## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [1]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='/Users/jordanharris/Code/mlops_zoomcamp/mlruns/2', creation_time=1752129813360, experiment_id='2', last_update_time=1752129813360, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='/Users/jordanharris/Code/mlops_zoomcamp/mlruns/1', creation_time=1751031231135, experiment_id='1', last_update_time=1751031231135, lifecycle_stage='active', name='nyc_taxi_trip_duration_experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1751031218635, experiment_id='0', last_update_time=1751031218635, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/07/10 11:53:11 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
2025/07/10 11:53:12 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/10 11:53:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 'mlflow-artifacts:/3/873242c02e8545a880389e6c3f63bead/artifacts'
🏃 View run mercurial-yak-419 at: http://127.0.0.1:5000/#/experiments/3/runs/873242c02e8545a880389e6c3f63bead
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3


In [5]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/3', creation_time=1752141192037, experiment_id='3', last_update_time=1752141192037, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='/Users/jordanharris/Code/mlops_zoomcamp/mlruns/2', creation_time=1752129813360, experiment_id='2', last_update_time=1752129813360, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='/Users/jordanharris/Code/mlops_zoomcamp/mlruns/1', creation_time=1751031231135, experiment_id='1', last_update_time=1751031231135, lifecycle_stage='active', name='nyc_taxi_trip_duration_experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1751031218635, experiment_id='0', last_update_time=1751031218635, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [6]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [7]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1751036829255, deployment_job_id='', deployment_job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', description='', last_updated_timestamp=1752126810797, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1752126810797, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description=None, last_updated_timestamp=1752126810797, metrics=None, model_id=None, name='lasso_regression', params=None, run_id='768c78e01f7449949577bbb86b3ac1dc', run_link='', source='models:/m-0ae9bb57705f41c99657e790d78e1a55', status='READY', status_message=None, tags={}, user_id='', version='4'>], name='lasso_regression', tags={'model': 'sklearn'}>,
 <RegisteredModel: aliases={}, creation_timestamp=1751036820963, deployment_job_id='', deployment_job_state='DEPLOYMENT_JOB_CON

In [8]:
run_id = client.search_runs(experiment_ids='1')[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.


MlflowException: Unable to find a logged_model with artifact_path models under run 811380bf278d4385b300e21c6d6d05cc